In [9]:
import sys
sys.path.insert(1, '/Users/juliengadonneix/Desktop/stage3a/Models_EEG_EmotionRecognition')

from models.EEGModels import EEGNet, ShallowConvNet, DeepConvNet
import torch
import os

In [10]:
model = EEGNet(nb_classes = 5, Chans = 14, Samples = 128, dropoutRate = 0.1, kernLength = 16, F1 = 64, D = 8, F2 = 64, dropoutType = 'Dropout')

total_params = sum(p.numel() for p in model.parameters())
print("Total number of parameters: ", total_params)

Total number of parameters:  51717


In [11]:
print(os.cpu_count())
print(torch.cuda.device_count()) 

8
0
